# Домашнее задание №9. Transformers

In [1]:
#!pip install transformers

In [2]:
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam
from tqdm import tqdm
from collections import Counter
import torchmetrics
import matplotlib.pyplot as plt

import pandas as pd
import transformers
from transformers import pipeline
from transformers import BertTokenizer, BertForSequenceClassification

## Задание

1. Возьмите готовую модель из https://huggingface.co/models для классификации сентимента текста.

2. Сделайте предсказания на всем df_val. Посчитайте метрику качества.

3. Дообучите эту модель на df_train. Посчитайте метрику качества на df_val.

## План работы

[0. Модель классификации сентимента текста 'SkolkovoInstitute/russian_toxicity_classifier'](#section_0)

[1. Загрузка данных](#section_1)

[2. Dataset и Dataloader](#section_2)
    
[3. Метрика модели 'SkolkovoInstitute/russian_toxicity_classifier' на валидационном датасете](#section_3)

[4. Дообученная модель 'SkolkovoInstitute/russian_toxicity_classifier' и ее метрики](#section_4)

[5. Выводы](#section_5)

### 0. Модель классификации сентимента текста 'SkolkovoInstitute/russian_toxicity_classifier'  <a id='section_0'></a>

**Описание классификатора SkolkovoInstitute/russian_toxicity_classifier**

Bert-based classifier (finetuned from [Conversational Rubert](https://huggingface.co/DeepPavlov/rubert-base-cased-conversational)) trained on merge of Russian Language Toxic Comments [dataset](https://www.kaggle.com/blackmoon/russian-language-toxic-comments/metadata) collected from 2ch.hk and Toxic Russian Comments [dataset](https://www.kaggle.com/alexandersemiletov/toxic-russian-comments) collected from ok.ru.

The datasets were merged, shuffled, and split into train, dev, test splits in 80-10-10 proportion.
The metrics obtained from test dataset is as follows

|              | precision | recall | f1-score | support |
|:------------:|:---------:|:------:|:--------:|:-------:|
|       0      |    0.98   |  0.99  |   0.98   | 21384   |
|       1      |    0.94   |  0.92  |   0.93   | 4886    |
|   accuracy   |       |   |   0.97  |         26270|
| macro avg    | 0.96      | 0.96   | 0.96     | 26270   |
| weighted avg | 0.97      | 0.97   | 0.97     | 26270   |


In [42]:
# загрузка модели
model_bert = BertForSequenceClassification.from_pretrained('SkolkovoInstitute/russian_toxicity_classifier')
print(model_bert)
print("Parameters full train:", sum([param.nelement() for param in model_bert.parameters()]))

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elemen

In [43]:
#пример классификации сентимента текста
sentiment = pipeline("text-classification", model='SkolkovoInstitute/russian_toxicity_classifier')
sentiment("Этот ресторан отличный")

[{'label': 'neutral', 'score': 0.998418927192688}]

In [44]:
#пример работы токенизации
tokenizer = BertTokenizer.from_pretrained('SkolkovoInstitute/russian_toxicity_classifier')

example_text = 'Пример текста для токенизации'

bert_input = tokenizer(example_text, padding='max_length', max_length=10, 
                       truncation=True, return_tensors="pt")


print(bert_input['input_ids'])
print(bert_input['attention_mask'])

tensor([[  101, 17863, 10316,   949, 15427,   831,  2703,   102,     0,     0]])
tensor([[1, 1, 1, 1, 1, 1, 1, 1, 0, 0]])


In [45]:
tokenizer.ids_to_tokens[101], tokenizer.ids_to_tokens[17863], tokenizer.ids_to_tokens[10316]

('[CLS]', 'Пример', 'текста')

In [46]:
tokenizer.get_vocab()

{'[PAD]': 0,
 '[unused1]': 1,
 '[unused2]': 2,
 '[unused3]': 3,
 '[unused4]': 4,
 '[unused5]': 5,
 '[unused6]': 6,
 '[unused7]': 7,
 '[unused8]': 8,
 '[unused9]': 9,
 '[unused10]': 10,
 '[unused11]': 11,
 '[unused12]': 12,
 '[unused13]': 13,
 '[unused14]': 14,
 '[unused15]': 15,
 '[unused16]': 16,
 '[unused17]': 17,
 '[unused18]': 18,
 '[unused19]': 19,
 '[unused20]': 20,
 '[unused21]': 21,
 '[unused22]': 22,
 '[unused23]': 23,
 '[unused24]': 24,
 '[unused25]': 25,
 '[unused26]': 26,
 '[unused27]': 27,
 '[unused28]': 28,
 '[unused29]': 29,
 '[unused30]': 30,
 '[unused31]': 31,
 '[unused32]': 32,
 '[unused33]': 33,
 '[unused34]': 34,
 '[unused35]': 35,
 '[unused36]': 36,
 '[unused37]': 37,
 '[unused38]': 38,
 '[unused39]': 39,
 '[unused40]': 40,
 '[unused41]': 41,
 '[unused42]': 42,
 '[unused43]': 43,
 '[unused44]': 44,
 '[unused45]': 45,
 '[unused46]': 46,
 '[unused47]': 47,
 '[unused48]': 48,
 '[unused49]': 49,
 '[unused50]': 50,
 '[unused51]': 51,
 '[unused52]': 52,
 '[unused53]': 53

In [47]:
example_text = tokenizer.decode(bert_input.input_ids[0])

print(example_text)

[CLS] Пример текста для токенизации [SEP] [PAD] [PAD]


### 1. Загрузка данных  <a id='section_1'></a>

**Загрузим и просмотрим данные из тренировочного и валидационного датасетов**

In [48]:
df_train = pd.read_csv("data/transforms/train.csv")
df_train.head()

,id,text,class
0,0,@alisachachka не уезжаааааааай. :(❤ я тоже не ...,0
1,1,RT @GalyginVadim: Ребята и девчата!\nВсе в кин...,1
2,2,RT @ARTEM_KLYUSHIN: Кто ненавидит пробки ретви...,0
3,3,RT @epupybobv: Хочется котлету по-киевски. Зап...,1
4,4,@KarineKurganova @Yess__Boss босапопа есбоса н...,1


In [49]:
df_val = pd.read_csv("data/transforms/val.csv")
df_val.head()

,id,text,class
0,181467,RT @TukvaSociopat: Максимальный репост! ))) #є...,1
1,181468,чтоб у меня з.п. ежегодно индексировали на инд...,0
2,181469,@chilyandlime нехуя мне не хорошо !!! :((((,0
3,181470,"@inafish нее , когда ногами ахахах когда?ахаха...",0
4,181471,"Хочу сделать как лучше, а получаю как всегда. :(",0


In [50]:
sentiment = pipeline("text-classification", model='SkolkovoInstitute/russian_toxicity_classifier')

idx = 0
print(df_train.iloc[idx]['text'])
print('label is', df_train.iloc[idx]['class'])
print('label by model is', sentiment(df_train.iloc[idx]['text'])[0]['label'], 'with score', sentiment(df_train.iloc[idx]['text'])[0]['score'])

@alisachachka не уезжаааааааай. :(❤ я тоже не хочу, чтобы ты уезжала.
label is 0
label by model is neutral with score 0.9937475919723511


### 2. Dataset и Dataloader <a id='section_2'></a>

**Создадим датасет и далаоадер для загруженных данных**

In [12]:
class TwitterDataset(torch.utils.data.Dataset):
    
    def __init__(self, txts, labels):
        self._labels = labels
        
        self.tokenizer = BertTokenizer.from_pretrained('SkolkovoInstitute/russian_toxicity_classifier')
        #для каждого text возвращает батч с полями:
               #'inputs_ids' -- тензор размера (B,1,max_len) из id токенов
               #'token_type_ids' -- тензор размера (B,1,max_len) из id типов токенов
               #'attention_mask' -- тензор размера (B,1,max_len) из индексов, указывающих, на какие токеты модель должна обратить внима
        self._txts = [self.tokenizer(text, padding='max_length', max_length=10,
                                     truncation=True, return_tensors="pt")
                      for text in txts]
        
    def __len__(self):
        return len(self._txts)
    
    def __getitem__(self, index):
        return self._txts[index], self._labels[index]

In [13]:
y_train = df_train['class'].values
y_val = df_val['class'].values

train_dataset = TwitterDataset(df_train['text'], y_train)
valid_dataset = TwitterDataset(df_val['text'], y_val)

train_loader = torch.utils.data.DataLoader(train_dataset,
                          batch_size=64,
                          shuffle=True,
                          num_workers=0)
valid_loader = torch.utils.data.DataLoader(valid_dataset,
                          batch_size=64,
                          shuffle=False,
                          num_workers=0)

In [14]:
for txt, lbl in train_loader:
    print(txt.keys()) #словарь с ключами'input_ids', 'token_type_ids', 'attention_mask'
    print(txt['input_ids'].shape) #тензор размера (B,1,max_len) из id токенов
    print(txt['attention_mask'].shape) #тензор размера (B,1,max_len) из индексов, указывающих, на какие токеты модель должна обратить внимание
    break

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])
torch.Size([64, 1, 10])
torch.Size([64, 1, 10])


### 3. Метрика модели 'SkolkovoInstitute/russian_toxicity_classifier' на валидационном датасете <a id='section_3'></a>

**Вычислим метрику f1_score модели SkolkovoInstitute/russian_toxicity_classifier на валидационном датасете**

In [41]:
#метрика модели на валидационном датасете
model_bert = BertForSequenceClassification.from_pretrained('SkolkovoInstitute/russian_toxicity_classifier')

valid_f1 = torchmetrics.F1Score()

for val_input, val_label in valid_loader:
    val_label = val_label
    mask = val_input['attention_mask'] 
    input_id = val_input['input_ids'].squeeze(1)
    
    output = model_bert(input_id, mask)[0]
    
    valid_f1(output, val_label)
    
print(f'Val f1_score: {valid_f1.compute().item():.3f}')

Val f1_score: 0.486


### 4. Дообученная модель 'SkolkovoInstitute/russian_toxicity_classifier' и ее метрики <a id='section_4'></a>

**Дообучим слой classifier модели SkolkovoInstitute/russian_toxicity_classifier и вычислим ее метрики на тренировочном и валидационном датасетах**

In [33]:
#инициализация модели классификации сентимента текста
model = BertForSequenceClassification.from_pretrained('SkolkovoInstitute/russian_toxicity_classifier')
print(model)
print("Parameters full train:", sum([param.nelement() for param in model.parameters()]))
print("Parameters transfer learning:", sum([param.nelement() for param in model.classifier.parameters()]))

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elemen

In [34]:
#компиляция модели
criterion = nn.CrossEntropyLoss()
# обучаем последний слой классификации
optimizer = Adam(model.classifier.parameters(), lr=0.001)  

In [36]:
#дообучение модели и подсчет метрик на тренировочном и валидационном датасетах

train_f1 = torchmetrics.F1Score()
valid_f1 = torchmetrics.F1Score()

epochs = 5

for epoch in range(epochs):
    model.train()
    total_loss_train = 0.0
    for train_input, train_label in tqdm(train_loader):
        mask = train_input['attention_mask']
        input_id = train_input['input_ids'].squeeze(1)
        train_label = train_label

        output = model(input_id, mask)[0]
                
        batch_loss = criterion(output, train_label)
        total_loss_train += batch_loss.item()
                
        train_f1(output, train_label)
        
        model.zero_grad()
        batch_loss.backward()
        optimizer.step()
            
    model.eval()
    total_loss_val = 0.0
    for val_input, val_label in valid_loader:
        val_label = val_label
        mask = val_input['attention_mask']
        input_id = val_input['input_ids'].squeeze(1)

        output = model(input_id, mask)[0]

        batch_loss = criterion(output, val_label)
        total_loss_val += batch_loss.item()
                    
        valid_f1(output, val_label)
            
    print(
        f'Epochs: {epoch + 1} | Train Loss: {total_loss_train / len(train_dataset): .3f} \
        | Train f1: {train_f1.compute().item(): .3f} \
        | Val Loss: {total_loss_val / len(valid_dataset): .3f} \
        | Val f1: {valid_f1.compute().item(): .3f}')
    train_f1.reset()
    valid_f1.reset()

  0%|                                                                                         | 0/2836 [00:00<?, ?it/s]

Epochs: 1 | Train Loss:  0.011         | Train f1:  0.582         | Val Loss:  0.010         | Val f1:  0.615


  0%|                                                                                         | 0/2836 [00:00<?, ?it/s]

Epochs: 2 | Train Loss:  0.010         | Train f1:  0.594         | Val Loss:  0.011         | Val f1:  0.577


  0%|                                                                                         | 0/2836 [00:00<?, ?it/s]

Epochs: 3 | Train Loss:  0.010         | Train f1:  0.595         | Val Loss:  0.010         | Val f1:  0.623


  0%|                                                                                         | 0/2836 [00:00<?, ?it/s]

Epochs: 4 | Train Loss:  0.010         | Train f1:  0.594         | Val Loss:  0.010         | Val f1:  0.615


100%|████████████████████████████████████████████████████████████████████████████| 2836/2836 [1:50:28<00:00,  2.34s/it]


Epochs: 5 | Train Loss:  0.010         | Train f1:  0.596         | Val Loss:  0.010         | Val f1:  0.620
Wall time: 9h 32min 40s


In [38]:
#метрика дообученной модели на валидационном датасете
valid_f1 = torchmetrics.F1Score()
model.eval()

for val_input, val_label in valid_loader:
    val_label = val_label
    mask = val_input['attention_mask'] 
    input_id = val_input['input_ids'].squeeze(1)
    
    output = model(input_id, mask)[0]
    
    valid_f1(output, val_label)
    
print(f'Val f1_score: {valid_f1.compute().item():.3f}')

Val f1_score: 0.620


**Добавим к выходу модели SkolkovoInstitute/russian_toxicity_classifier сигмоиду, дообучим последний слой классификации и вычислим ее метрики на тренировочном и валидационном датасетах**

In [56]:
#класс модели BertClassifier
class BertClassifier(nn.Module):

    def __init__(self, dropout=0.5):
        super().__init__()
        self.bert = BertForSequenceClassification.from_pretrained('SkolkovoInstitute/russian_toxicity_classifier')
        self.sigm = nn.Sigmoid()

    def forward(self, x, mask):
        pooled_output = self.bert(input_ids=x, attention_mask=mask,return_dict=False)[0]  #(B, 2)
        final_layer = self.sigm(pooled_output)
        return final_layer

In [57]:
#инициализация модели
model_sigm = BertClassifier()
print(model_sigm)
print("Parameters full train:", sum([param.nelement() for param in model_sigm.parameters()]))
print("Parameters transfer learning:", sum([param.nelement() for param in model_sigm.bert.classifier.parameters()]))

BertClassifier(
  (bert): BertForSequenceClassification(
    (bert): BertModel(
      (embeddings): BertEmbeddings(
        (word_embeddings): Embedding(119547, 768, padding_idx=0)
        (position_embeddings): Embedding(512, 768)
        (token_type_embeddings): Embedding(2, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): BertEncoder(
        (layer): ModuleList(
          (0): BertLayer(
            (attention): BertAttention(
              (self): BertSelfAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, bias=True)
                (dropout): Dropout(p=0.1, inplace=False)
              )
              (output): BertSelfOutput(
                (dense): Linear(in_features=768, out_features=768, bias=Tr

In [58]:
#компиляция модели
criterion = nn.CrossEntropyLoss()
#дообучаем последний слой классификации
optimizer = Adam(model_sigm.bert.classifier.parameters(), lr=0.001)

In [65]:
#дообучение модели и подсчет метрик на тренировочном и валидационном датасетах

train_f1 = torchmetrics.F1Score()
valid_f1 = torchmetrics.F1Score()

epochs = 5

for epoch in range(epochs):
    model_sigm.train()
    total_loss_train = 0.0
    for train_input, train_label in tqdm(train_loader):
        mask = train_input['attention_mask']
        input_id = train_input['input_ids'].squeeze(1)
        train_label = train_label

        output = model_sigm(input_id, mask)
                
        batch_loss = criterion(output, train_label)
        total_loss_train += batch_loss.item()
                
        train_f1(output, train_label)
        
        model_sigm.zero_grad()
        batch_loss.backward()
        optimizer.step()
            
    model_sigm.eval()
    total_loss_val = 0.0
    for val_input, val_label in valid_loader:
        val_label = val_label
        mask = val_input['attention_mask']
        input_id = val_input['input_ids'].squeeze(1)

        output = model_sigm(input_id, mask)

        batch_loss = criterion(output, val_label)
        total_loss_val += batch_loss.item()
                    
        valid_f1(output, val_label)
            
    print(
        f'Epochs: {epoch + 1} | Train Loss: {total_loss_train / len(train_dataset): .3f} \
        | Train f1: {train_f1.compute().item(): .3f} \
        | Val Loss: {total_loss_val / len(valid_dataset): .3f} \
        | Val f1: {valid_f1.compute().item(): .3f}')
    train_f1.reset()
    valid_f1.reset()

  0%|                                                                                         | 0/2836 [00:00<?, ?it/s]

Epochs: 1 | Train Loss:  0.011         | Train f1:  0.586         | Val Loss:  0.010         | Val f1:  0.598


  0%|                                                                                         | 0/2836 [00:00<?, ?it/s]

Epochs: 2 | Train Loss:  0.010         | Train f1:  0.603         | Val Loss:  0.010         | Val f1:  0.606


  0%|                                                                                         | 0/2836 [00:00<?, ?it/s]

Epochs: 3 | Train Loss:  0.010         | Train f1:  0.607         | Val Loss:  0.010         | Val f1:  0.607


  0%|                                                                                         | 0/2836 [00:00<?, ?it/s]

Epochs: 4 | Train Loss:  0.010         | Train f1:  0.608         | Val Loss:  0.010         | Val f1:  0.613


100%|████████████████████████████████████████████████████████████████████████████| 2836/2836 [1:46:35<00:00,  2.26s/it]


Epochs: 5 | Train Loss:  0.010         | Train f1:  0.609         | Val Loss:  0.010         | Val f1:  0.615


In [66]:
#метрика дообученной модели на валидационном датасете
valid_f1 = torchmetrics.F1Score()
model_sigm.eval()

for val_input, val_label in valid_loader:
    val_label = val_label
    mask = val_input['attention_mask'] 
    input_id = val_input['input_ids'].squeeze(1)
    
    output = model_sigm(input_id, mask)
    
    valid_f1(output, val_label)
    
print(f'Val f1_score: {valid_f1.compute().item():.3f}')

Val f1_score: 0.615


### 5. Выводы <a id='section_5'></a>

В настоящей работе была рассмотрена модель классификации сантимента текста SkolkovoInstitute/russian_toxicity_classifier. 

Метрика f1_score модели на валидационном датасете:
- до дообучения -- 0.486, 
- после дообучения -- 0.620,
- после дообучения с применением сигмоиды -- 0.615.

Таким образом, дообучение модели на тренировочном датасете способствует увеличению значения метрики, при этом использование логитов дает лучший результат по сравнению с применением функции активации сигмоиды на последнем слое классификации. 

P.S. Отметим также, что дообучение модели занимает большое количество времени (около 2 часов на эпоху), что объясняет выбор числа эпох равного 5 в настоящей работе.